# CS246 - Colab 9
## Studying COVID-19

In [1]:
from IPython.display import Image
print("Colab 9 Mascot saying Bye-Bye :( \n")
Image(url='https://media.giphy.com/media/tuvMgAPzxaQBq/giphy.gif',width=200)

Colab 9 Mascot saying Bye-Bye :( 



### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=a5f37f767cbf47dd06a6a569e93c1fd4c5c123d935c48cd967ea2c667b728555
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1YT7ttUAafCjbVdm6obeHp1TWAK0rEtoR'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_confirmed_global.csv')

id='1YxEA5UQ2EFJ_9oLssM__Gs1ncVNufGNA'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_deaths_global.csv')

id='1CNxszuZTeIw-5cF5yrzKMZdb1qV0hSoy'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_recovered_global.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be analyzing the time series data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the cumulative number of confirmed COVID-19 cases, divided by geographical area
*   ```deaths```: dataframe containing the cumulative number of deaths due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the cumulative number of recoevered patients, divided by geographical area

The data sets contain data entries for each day, representing the cumulative totals as of that day.







In [7]:
confirmed = spark.read.csv('time_series_covid19_confirmed_global.csv', header=True)
deaths = spark.read.csv('time_series_covid19_deaths_global.csv', header=True)
recovered = spark.read.csv('time_series_covid19_recovered_global.csv', header=True)

In [128]:
import pyspark.pandas as ps

df1 = ps.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]},
                   columns=['lkey', 'value'])
df2 = ps.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]},
                   columns=['rkey', 'value'])

In [129]:
df1, df2

(  lkey  value
 0  foo      1
 1  bar      2
 2  baz      3
 3  foo      5,
   rkey  value
 0  foo      5
 1  bar      6
 2  baz      7
 3  foo      8)

In [132]:
df1.merge(df2, left_on= 'lkey', right_on= 'rkey')

,lkey,value_x,rkey,value_y
0,bar,2,bar,6
1,baz,3,baz,7
2,foo,1,foo,5
3,foo,1,foo,8
4,foo,5,foo,5
5,foo,5,foo,8


In [113]:
deaths.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- 1/22/20: string (nullable = true)
 |-- 1/23/20: string (nullable = true)
 |-- 1/24/20: string (nullable = true)
 |-- 1/25/20: string (nullable = true)
 |-- 1/26/20: string (nullable = true)
 |-- 1/27/20: string (nullable = true)
 |-- 1/28/20: string (nullable = true)
 |-- 1/29/20: string (nullable = true)
 |-- 1/30/20: string (nullable = true)
 |-- 1/31/20: string (nullable = true)
 |-- 2/1/20: string (nullable = true)
 |-- 2/2/20: string (nullable = true)
 |-- 2/3/20: string (nullable = true)
 |-- 2/4/20: string (nullable = true)
 |-- 2/5/20: string (nullable = true)
 |-- 2/6/20: string (nullable = true)
 |-- 2/7/20: string (nullable = true)
 |-- 2/8/20: string (nullable = true)
 |-- 2/9/20: string (nullable = true)
 |-- 2/10/20: string (nullable = true)
 |-- 2/11/20: string (nullable = true)
 |-- 2/12/20: string (

### Your Task

We are aware of the stress we are all experiencing because of the still-ongoing pandemic and the fact that many of you have projects and exams due this week. As such, we decided to conclude our series of Colabs with a **lightweight task** -- given everything you have learned about Spark during the quarter, this Colab should take you just a few minutes to complete.

Consider the entries for May 1, 2021, in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 (across the globe)
*   number of recovered patients across the globe



In [74]:
''' 6 lines of code in total expected. '''

# YOUR CODE HERE
num_conf = confirmed.select(sum('5/1/21')).collect()[0][0]
num_deaths = deaths.select(sum('5/1/21')).collect()[0][0]
num_rec = recovered.select(sum('5/1/21')).collect()[0][0]

print(f"number of confirmed cases: {num_conf}")
print(f"number of deaths: {num_deaths}")
print(f"number of recovered: {num_rec}")

number of confirmed cases: 152196159.0
number of deaths: 3192930.0
number of recovered: 88919401.0


Consider the data points for March 1, 2020, and March 1, 2021, and filter out the geographical locations where less than 50 cases have been confirmed.
For the areas still taken into consideration after the filtering step, compute the ratio between number of deaths and number of confirmed cases.

In [194]:
deaths_grp = deaths.groupby(['Province/State', 'Country/Region']).agg(sum('3/1/20').alias("deaths_20"),
                                                                      sum('3/1/21').alias("deaths_21"))
deaths_grp.show()

+--------------+--------------+---------+---------+
|Province/State|Country/Region|deaths_20|deaths_21|
+--------------+--------------+---------+---------+
|    Martinique|        France|      0.0|     45.0|
|          null|          Chad|      0.0|    140.0|
|         Gansu|         China|      2.0|      2.0|
| Faroe Islands|       Denmark|      0.0|      1.0|
|          null|      Paraguay|      0.0|   3198.0|
|          null|        Russia|      0.0|  85025.0|
|  Heilongjiang|         China|     13.0|     13.0|
|          null|         Yemen|      0.0|    634.0|
|          null|       Senegal|      0.0|    880.0|
|          null|    Cabo Verde|      0.0|    147.0|
|          null|        Sweden|      0.0|  12826.0|
|          null|        Guyana|      0.0|    197.0|
|          null|         Burma|      0.0|   3199.0|
|          null|       Eritrea|      0.0|      7.0|
|          null|   Philippines|      1.0|  12322.0|
|          null|      Djibouti|      0.0|     63.0|
|          n

In [195]:
''' 20-25 lines of code in total expected but can differ based on your style. '''
''' For sub-parts of the question (if any), creating different cells of code would be recommended.'''

# YOUR CODE HERE
conf_grp = confirmed.groupby(['Province/State', 'Country/Region']).agg(sum('3/1/20').alias("conf_20"),
                                                                          sum('3/1/21').alias("conf_21"))
conf_grp_filter = conf_grp.filter((col('conf_20') > 50) & (col('conf_21') > 50))
conf_grp_filter.show()

+--------------+--------------+-------+---------+
|Province/State|Country/Region|conf_20|  conf_21|
+--------------+--------------+-------+---------+
|         Gansu|         China|   91.0|    187.0|
|  Heilongjiang|         China|  480.0|   1610.0|
|          null|     Singapore|  106.0|  59948.0|
|        Fujian|         China|  296.0|    551.0|
|          null|       Germany|  130.0|2455569.0|
|      Shanghai|         China|  337.0|   1797.0|
|          null|        France|  130.0|3736390.0|
|     Guangdong|         China| 1349.0|   2215.0|
|        Hainan|         China|  168.0|    171.0|
|        Shanxi|         China|  133.0|    240.0|
|     Chongqing|         China|  576.0|    591.0|
|       Beijing|         China|  413.0|   1049.0|
|         Hebei|         China|  318.0|   1317.0|
|      Liaoning|         China|  122.0|    406.0|
|         Henan|         China| 1272.0|   1305.0|
|       Guangxi|         China|  252.0|    267.0|
|          null|         Italy| 1694.0|2938371.0|


In [155]:
conf_grp_filter.count()

40

In [196]:
conf_deaths_join = conf_grp_filter.join(deaths_grp, ['Province/State', 'Country/Region']).select(col('Province/State'), col('Country/Region'),\
                                                                                             col('conf_20'), col('conf_21'),\
                                                                                             col('deaths_20'), col('deaths_21'))

conf_deaths_join.show()

+--------------+--------------+-------+-------+---------+---------+
|Province/State|Country/Region|conf_20|conf_21|deaths_20|deaths_21|
+--------------+--------------+-------+-------+---------+---------+
|         Gansu|         China|   91.0|  187.0|      2.0|      2.0|
|  Heilongjiang|         China|  480.0| 1610.0|     13.0|     13.0|
|        Fujian|         China|  296.0|  551.0|      1.0|      1.0|
|      Shanghai|         China|  337.0| 1797.0|      3.0|      7.0|
|     Guangdong|         China| 1349.0| 2215.0|      7.0|      8.0|
|        Hainan|         China|  168.0|  171.0|      5.0|      6.0|
|        Shanxi|         China|  133.0|  240.0|      0.0|      0.0|
|     Chongqing|         China|  576.0|  591.0|      6.0|      6.0|
|       Beijing|         China|  413.0| 1049.0|      8.0|      9.0|
|         Hebei|         China|  318.0| 1317.0|      6.0|      7.0|
|      Liaoning|         China|  122.0|  406.0|      1.0|      2.0|
|         Henan|         China| 1272.0| 1305.0| 

In [197]:
conf_deaths_ratio = conf_deaths_join.withColumn('death_ratio_20', round(col('deaths_20')/col('conf_20'), 4)).\
                                     withColumn('death_ratio_21', round(col('deaths_20')/col('conf_21'), 4))
conf_deaths_ratio.show()

+--------------+--------------+-------+-------+---------+---------+--------------+--------------+
|Province/State|Country/Region|conf_20|conf_21|deaths_20|deaths_21|death_ratio_20|death_ratio_21|
+--------------+--------------+-------+-------+---------+---------+--------------+--------------+
|         Gansu|         China|   91.0|  187.0|      2.0|      2.0|         0.022|        0.0107|
|  Heilongjiang|         China|  480.0| 1610.0|     13.0|     13.0|        0.0271|        0.0081|
|        Fujian|         China|  296.0|  551.0|      1.0|      1.0|        0.0034|        0.0018|
|      Shanghai|         China|  337.0| 1797.0|      3.0|      7.0|        0.0089|        0.0017|
|     Guangdong|         China| 1349.0| 2215.0|      7.0|      8.0|        0.0052|        0.0032|
|        Hainan|         China|  168.0|  171.0|      5.0|      6.0|        0.0298|        0.0292|
|        Shanxi|         China|  133.0|  240.0|      0.0|      0.0|           0.0|           0.0|
|     Chongqing|    

Consider the data points for March 1, 2021, and May 1, 2021, in the timeseries, and filter out the geographical locations where less than 50 deaths have been confirmed (as of March 1, 2021).
For the areas still taken into consideration after the filtering step, compute the percent increase in cumulative deaths between the two dates.

In [213]:
''' 4-5 lines of code in total expected. '''

# YOUR CODE HERE
deaths_21 = deaths.groupby(['Province/State', 'Country/Region']).agg(sum('3/1/21').alias("deaths_mar"),
                                                                     sum('5/1/21').alias("deaths_may"))
conf_21 = confirmed.groupby(['Province/State', 'Country/Region']).agg(sum('3/1/21').alias("conf_mar"),
                                                                      sum('5/1/21').alias("conf_may"))

conf_21_filter = conf_21.filter((col('conf_may') > 50) & (col('conf_mar') > 50))

conf_death_21_ratio = conf_21_filter.join(deaths_21, ['Province/State', 'Country/Region'])

conf_death_21_ratio = conf_death_21_ratio.withColumn('ratio_mar', round(col('deaths_mar')/col('conf_mar'), 4))\
                                          .withColumn('ratio_may', round(col('deaths_may')/col('conf_may'), 4))

conf_death_21_ratio.show()

+--------------------+--------------+--------+--------+----------+----------+---------+---------+
|      Province/State|Country/Region|conf_mar|conf_may|deaths_mar|deaths_may|ratio_mar|ratio_may|
+--------------------+--------------+--------+--------+----------+----------+---------+---------+
|          Martinique|        France|  6746.0| 10491.0|      45.0|      70.0|   0.0067|   0.0067|
|               Gansu|         China|   187.0|   193.0|       2.0|       2.0|   0.0107|   0.0104|
|       Faroe Islands|       Denmark|   658.0|   664.0|       1.0|       1.0|   0.0015|   0.0015|
|        Heilongjiang|         China|  1610.0|  1610.0|      13.0|      13.0|   0.0081|   0.0081|
|              Fujian|         China|   551.0|   593.0|       1.0|       1.0|   0.0018|   0.0017|
|    French Polynesia|        France| 18414.0| 18758.0|     139.0|     141.0|   0.0075|   0.0075|
|Australian Capita...|     Australia|   118.0|   124.0|       3.0|       3.0|   0.0254|   0.0242|
|     New South Wale

Once you have working code for each cell above, **head over to Gradescope, read the questions carefully, and submit your solution for this Colab**!